In [5]:
import pandas as pd
import numpy as np
from pandas import DataFrame # specify datatype while passing df object

In [6]:
 #df = pd.read_csv('WMT.csv')
    #Uncomment this and read the WMT csv file wherever its stored on your local device

In [8]:
#This is where you will code up the technical indicators and add as columns in the original dataframe

def rsi(df):
    Difference = pd.Series(df['Close'].diff())
    Gain = pd.Series(np.where(Difference>0,Difference,0))
    Loss = pd.Series(np.where(Difference<0,-Difference,0))
    for period in range(6,26):
      AvgGain = pd.Series(Gain.rolling(window=period).mean())
      AvgLoss = pd.Series(Loss.rolling(window=period).mean())
      RS=pd.Series(AvgGain/AvgLoss)
      df[f'RSI_{period}']=pd.Series(100-100/(1+RS))
    return df
    
def cmo(df):
    Difference = pd.Series(df["Close"] - df["Close"].shift(1))
    Sum=pd.Series(Difference.abs())
    for period in range(6, 21):       
        diff = Difference.rolling(window=period).sum() 
        sum = Sum.rolling(window=period).sum()
        df[f'cmo_{period}'] = (diff/sum) * 100 
    return df
    
def williams(df):
   for period in range(6,21):
     lowest_low=pd.Series(df["Low"].rolling(window = period).min())
     highest_high=pd.Series(df["High"].rolling(window = period).max())  
     df[f'Williams_{period}']=pd.Series(((highest_high-df['Close'])/(highest_high-lowest_low))*-100)
   return df
    
def sma(df: DataFrame):
    '''
    SMA: Simple Moving Average; computes the average of a selected
    range of prices, over periods 6 through 30(both inclusive).
    
    Arguement: 
        df: DataFrame object initialised from the csv data
    Returns:
        df: Modified DataFrame object
    '''
    
    for period in range(6, 31):
        col_name = f'sma_{period}'
        df[col_name] = df["Adj Close"].rolling(window=period).mean()
    
    return df 
    
def wma(df: DataFrame):
    '''
    WMA: Weighted Moving Average; places a greater weight on the more recent
    datapoints. Calculated over periods 6 through 20(both inclusive).
    
    Arguement: 
        df: DataFrame object initialised from the csv data
    Returns:
        df: Modified DataFrame object
    '''
    for period in range(6, 21):
        col_name = f'wma_{period}'
        weights = np.arange(1, period+1)
        denominator = np.sum(weights)

        df[col_name] = df["Adj Close"].rolling(window=period).apply(
                        lambda x: np.sum(weights*x)/denominator)
    
    return df
    
def ppo(df):
    for period in range(6, 21):
        ema_lower = df["Adj Close"].ewm(span = period).mean()
        ema_higher = df["Adj Close"].ewm(span = period + 14).mean()
        df[f'ppo_{period}'] = (ema_lower - ema_higher)*100/ema_higher
        df[f'ppo_signal_{period}'] = df[f'ppo_{period}'].ewm(span=period - 3).mean()
    
    return df
    
def ema(df: DataFrame):
    '''
    EMA: Exponential Moving Average; similar to WMA, it places a greater weight on the recent
    datapoints and hence reacts more significantly to price changes than SMA.
    Calculated over periods 6 through 20(both inclusive).
    
    Arguement: 
        df: DataFrame object initialised from the csv data
    Returns:
        df: Modified DataFrame object
    '''
    for period in range(6, 21):
        col_name = f'ema_{period}'
        df[col_name] = df["Adj Close"].ewm(span=period, adjust=True).mean()

    return df
    
def roc():
    for n in range(6,21):
       col_name=f'roc_{n}'
       D = df['Adj Close'].diff(n)
       N= df['Adj Close'].shift(n)
       df[col_name] = (D/N)*100
   return df
    
def cmfi(df):
  MFM = ((df['Close']-df['Low']) - (df['High']-df['Close']))/(df['High']-df['Low'])      #Calculating Multiplier
  MFV = MFM * df['Volume']           
  for period in range(6,21):
    df[f'cmfi_{period}'] = (MFV.rolling(window = period).sum())/(df['Volume'].rolling(window = period).sum())
  return df

def hma(df):
    for len in range(6,21):
        col_name = f'hma_{len}'
        half_len = len//2
        k = int(np.sqrt(len))
        half_weights = np.arange(1, half_len+1)
        weights = np.arange(1, len+1)
        half_wma = df['Adj Close'].rolling(half_len).apply(lambda x: np.dot(x, half_weights)/half_weights.sum(), raw=True)
        full_wma = df['Adj Close'].rolling(len).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
        diff = (2*half_wma) - full_wma
        w = np.arange(1, k+1)
        df[col_name] = diff.rolling(k).apply(lambda x: np.dot(x, w)/w.sum(), raw=True)
    return df
    
def tripleema(df):
    for period in range(6,21):
        ema1 = df['Adj Close'].ewm(span=period).mean()
        ema2 = ema1.ewm(span=period).mean()
        ema3 = ema2.ewm(span=period).mean()
        df[f'tripleema_{period}'] = ((3*ema1) - (3*ema2) + ema3)
    return df
    
def psi(df):        #Psychology Line Index Indicator
    movement = pd.Series(np.where(df['Close']-df['Close'].shift(1) > 0 ,'UP',None))
    for period in range(6,21):
      df[f'psi_{period}']=((movement.rolling(window=period).count())/period)*100
    return df
    
def cci(df): 
    TP = (df['High'] + df['Low'] + df['Close']) / 3 
    for period in range(6,21):
      df[f'cci_{period}']= pd.Series((TP - TP.rolling(window = period).mean()) / (0.015 * TP.rolling(window = period).std()))
    return df

    


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 92)

In [ ]:
def label_data(df):
  """
  Research Paper Algorithm's of Labelling Method is simply telling us to take Window Size of 11 period and check,
  whether the middle number is maximum in the window, then Label it as "Sell", or
  if the middle number is minimum in the window, then Label it as "Buy",
  else Label it as "Hold", and then Slide the Window by 1 Period
  """
  """
  Parameter : df ==> Dataframe with OHLCV data.
  Returns : df ==> Dataframe with 1 added column of "Label".
  """
  df['Label'] = ''
  WindowSize = 11
  counter_row = 0

  while counter_row < len(df):
    if counter_row >= WindowSize:
      
      #Creating the Window
      WindowBeginIndex = counter_row - WindowSize             
      WindowEndIndex = WindowBeginIndex + WindowSize - 1
      WindowMidIndex = (WindowBeginIndex + WindowEndIndex)/2
      min = np.inf
      max = 0
      min_index = -1
      max_index = -1
      
      #Finding Maximum and minimum in the Window 
      for i in range (WindowBeginIndex, WindowEndIndex +1):   
        number = df['Close'][i]
        if number < min:
          min = number
          min_index = i
        if number > max:
          max = number
          max_index = i
      
      #Checking if the Middle number of Window is Max or Min and classifying them as "SELL","BUY","HOLD"
      if max_index == WindowMidIndex:                         
        df['Label'][WindowMidIndex] = 0       #SELL
      elif min_index == WindowMidIndex:
        df['Label'][WindowMidIndex] = 1       #BUY
      else :
        df['Label'][WindowMidIndex] = 2       #HOLD
    counter_row = counter_row + 1
  return df